In [5]:
!pip install SpeechRecognition

In [6]:
!pip install rasa_nlu

     |████████████████████████████████| 153kB 3.9MB/s 
     |████████████████████████████████| 51kB 5.6MB/s 
     |████████████████████████████████| 133kB 18.9MB/s 
     |████████████████████████████████| 133kB 38.1MB/s 
     |████████████████████████████████| 829kB 39.5MB/s 
     |████████████████████████████████| 655kB 28.5MB/s 
     |████████████████████████████████| 5.1MB 34.3MB/s 
     |████████████████████████████████| 12.8MB 26.3MB/s 
     |████████████████████████████████| 5.4MB 26.2MB/s 
     |████████████████████████████████| 81kB 8.1MB/s 
     |████████████████████████████████| 92kB 10.0MB/s 
     |████████████████████████████████| 3.1MB 36.0MB/s 
     |████████████████████████████████| 81kB 8.2MB/s 
     |████████████████████████████████| 7.6MB 31.7MB/s 
     |████████████████████████████████| 81kB 8.6MB/s 
     |████████████████████████████████| 256kB 44.5MB/s 
  Created wheel for future: filename=future-0.17.1-cp37-none-any.whl size=488742 sha256=af21107d3acedd83830ef125e

In [7]:
!pip install rasa_core

     |████████████████████████████████| 215kB 3.9MB/s 
     |████████████████████████████████| 51kB 6.1MB/s 
     |████████████████████████████████| 92.7MB 43kB/s 
     |████████████████████████████████| 40kB 4.3MB/s 
     |████████████████████████████████| 327kB 43.8MB/s 
     |████████████████████████████████| 122kB 42.7MB/s 
     |████████████████████████████████| 481kB 41.4MB/s 
     |████████████████████████████████| 61kB 7.0MB/s 
     |████████████████████████████████| 71kB 6.1MB/s 
     |████████████████████████████████| 112kB 31.3MB/s 
     |████████████████████████████████| 368kB 30.3MB/s 
     |████████████████████████████████| 61kB 5.0MB/s 
     |████████████████████████████████| 204kB 31.3MB/s 
     |████████████████████████████████| 368kB 26.7MB/s 
     |████████████████████████████████| 51kB 4.8MB/s 
     |████████████████████████████████| 3.2MB 22.8MB/s 
     |████████████████████████████████| 3.2MB 19.0MB/s 
     |████████████████████████████████| 61kB 5.9MB/s 
  Create

In [8]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [9]:
import requests
import json

In [10]:
print('-------Enter the link for the data to be tested--------')
L=input()

-------Enter the link for the data to be tested--------


KeyboardInterrupt: ignored

In [ ]:
request = requests.get(L)
D=request.json()


In [ ]:
D

{'iss_position': {'latitude': '12.0197', 'longitude': '112.3240'},
 'message': 'success',
 'timestamp': 1622900153}

In [11]:
D={"question_key": "q1","options":["Diabetes","Thyroid","Cancer"],"audio": '/content/drive/MyDrive/SpeechRecognition/1.wav'}

In [12]:
import speech_recognition as sr
def speech_text(audio):
  # Initialize recognizer class (for recognizing the speech)
  r = sr.Recognizer()
  # Reading Audio file as source
  # listening the audio file and store in audio_text variable
  with sr.AudioFile(audio) as source:
    audio_text = r.listen(source)
  # recoginize_() method will throw a request error if the API is unreachable, hence using exception handling
  try:
  # using google speech recognition
    text = r.recognize_google(audio_text)
  except:
    print('Sorry.. run again...')
  return text

In [13]:
audio=D["audio"]
text=speech_text(audio)

FileNotFoundError: ignored

In [ ]:
def q1(text,options):
  
  return answers

In [ ]:
q=D['question_key']
if q=='q1':
  options=D['options']
  answers=q1(text,options)



In [ ]:
q

'q1'

In [14]:
import rasa_nlu
import rasa_core
import spacy

/usr/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got

In [ ]:
import rasa_nlu
import rasa_core
import spacy

In [15]:
# Import modules for training
from rasa_nlu.training_data import load_data
from rasa_nlu.config import RasaNLUModelConfig
from rasa_nlu.model import Trainer
from rasa_nlu import config

In [ ]:
# loading the nlu training samples
training_data = load_data("nlu.md")
trainer = Trainer(config.load("config.yml"))

C:\Users\Saahithi\anaconda3\envs\env_py_36\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Saahithi\anaconda3\envs\env_py_36\lib\site-packages\tensorflow\python\framework\dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Saahithi\anaconda3\envs\env_py_36\lib\site-packages\tensorflow\python\framework\dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Saahithi\anaconda3\envs\env_py_36\lib\site-

In [ ]:
# training the nlu
interpreter = trainer.train(training_data)
model_directory = trainer.persist("./models/nlu", fixed_model_name="current")

Fitting 2 folds for each of 6 candidates, totalling 12 fits


[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:    0.0s finished


In [ ]:
import json
def pprint(o):   
    print(json.dumps(o, indent=2))

pprint(interpreter.parse("hi"))

{
  "intent": {
    "name": "greet",
    "confidence": 0.788972797463024
  },
  "entities": [],
  "intent_ranking": [
    {
      "name": "greet",
      "confidence": 0.788972797463024
    },
    {
      "name": "name",
      "confidence": 0.06020464242674477
    },
    {
      "name": "order",
      "confidence": 0.04057661533805654
    },
    {
      "name": "goodbye",
      "confidence": 0.03084105498789908
    },
    {
      "name": "choosing_item",
      "confidence": 0.024737553437997748
    },
    {
      "name": "suggest",
      "confidence": 0.023522003326863845
    },
    {
      "name": "complain",
      "confidence": 0.02100868364071376
    },
    {
      "name": "order_enquiry",
      "confidence": 0.01013664937869995
    }
  ],
  "text": "hi"
}


In [ ]:
# Import the policies and agent
from rasa_core.policies import FallbackPolicy, MemoizationPolicy,KerasPolicy
from rasa_core.agent import Agent

# Initialize the model with `domain.yml`
agent = Agent('domain.yml', policies=[MemoizationPolicy(), KerasPolicy()])

# loading our  training dialogues from `stories.md` 
training_data = agent.load_data('stories.md')


# Training the model
agent.train(
    training_data,
    validation_split=0.0,
    epochs=200
)

agent.persist('models/dialogue')

Using TensorFlow backend.
C:\Users\Saahithi\anaconda3\envs\env_py_36\lib\site-packages\rasa_core\utils.py:341: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  return yaml.load(read_file(filename, "utf-8"))
Processed trackers: 100%|██████████| 301/301 [00:02<00:00, 148.31it/s, # actions=451]
Processed actions: 451it [00:00, 1034.80it/s, # examples=451]
Processed trackers: 100%|██████████| 301/301 [00:02<00:00, 150.30it/s, # actions=451]


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking_1 (Masking)          (None, 5, 22)             0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                7040      
_________________________________________________________________
dense_1 (Dense)              (None, 10)                330       
_________________________________________________________________
activation_1 (Activation)    (None, 10)                0         
Total params: 7,370
Trainable params: 7,370
Non-trainable params: 0
_________________________________________________________________
Epoch 1/200
451/451 [==============================] - 1s 2ms/step - loss: 2.2075 - acc: 0.3792
Epoch 2/200
451/451 [==============================] - 0s 164us/step - loss: 2.0623 - acc: 0.5011
Epoch 3/200
451/451 [==============================] - 0s 161us/step - loss: 1

451/451 [==============================] - 0s 146us/step - loss: 0.1590 - acc: 0.9823
Epoch 75/200
451/451 [==============================] - 0s 137us/step - loss: 0.1346 - acc: 0.9867
Epoch 76/200
451/451 [==============================] - 0s 142us/step - loss: 0.1268 - acc: 0.9867
Epoch 77/200
451/451 [==============================] - 0s 133us/step - loss: 0.1188 - acc: 0.9867
Epoch 78/200
451/451 [==============================] - 0s 133us/step - loss: 0.0932 - acc: 0.9911
Epoch 79/200
451/451 [==============================] - 0s 135us/step - loss: 0.1187 - acc: 0.9867
Epoch 80/200
451/451 [==============================] - 0s 135us/step - loss: 0.1298 - acc: 0.9911
Epoch 81/200
451/451 [==============================] - 0s 133us/step - loss: 0.1296 - acc: 0.9845
Epoch 82/200
451/451 [==============================] - 0s 130us/step - loss: 0.0988 - acc: 0.9933
Epoch 83/200
451/451 [==============================] - 0s 139us/step - loss: 0.1022 - acc: 0.9911
Epoch 84/200
451/451 [=

451/451 [==============================] - 0s 139us/step - loss: 0.0440 - acc: 0.9978
Epoch 157/200
451/451 [==============================] - 0s 133us/step - loss: 0.0261 - acc: 1.0000
Epoch 158/200
451/451 [==============================] - 0s 135us/step - loss: 0.0255 - acc: 0.9956
Epoch 159/200
451/451 [==============================] - 0s 135us/step - loss: 0.0247 - acc: 0.9978
Epoch 160/200
451/451 [==============================] - 0s 135us/step - loss: 0.0416 - acc: 0.9911
Epoch 161/200
451/451 [==============================] - 0s 130us/step - loss: 0.0455 - acc: 0.9889
Epoch 162/200
451/451 [==============================] - 0s 135us/step - loss: 0.0113 - acc: 1.0000
Epoch 163/200
451/451 [==============================] - 0s 135us/step - loss: 0.0234 - acc: 0.9956
Epoch 164/200
451/451 [==============================] - 0s 137us/step - loss: 0.0389 - acc: 0.9889
Epoch 165/200
451/451 [==============================] - 0s 150us/step - loss: 0.0334 - acc: 0.9956
Epoch 166/200


In [ ]:
from rasa_core.agent import Agent
agent = Agent.load('models/dialogue', interpreter=model_directory)

C:\Users\Saahithi\anaconda3\envs\env_py_36\lib\site-packages\rasa_core\utils.py:341: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  return yaml.load(read_file(filename, "utf-8"))


In [ ]:
print("Your bot is ready to talk! Type your messages here or send 'stop'")
while True:
    a = input()
    if a == 'stop':
        break
    responses = agent.handle_message(a)
    for response in responses:
        print(response["text"])

Your bot is ready to talk! Type your messages here or send 'stop'
hi
Welcome to The Grocers! Please enter your name.
I am saahithi
Hello! How can I help you?
